In [1]:
import os
import librosa
# import pydub
from pydub import AudioSegment
import numpy as np
import copy
import random
from scipy import signal as scipy_signal
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim

import datetime

In [2]:
dataset_dir = '/home/torooc/dataHDD2/speech_commands_v0.01/'

In [3]:
testing_list_path = dataset_dir + 'testing_list.txt'
validation_list_path = dataset_dir + 'validation_list.txt'

In [4]:
with open(testing_list_path) as f:
    testing_list = f.readlines()
    testing_list = [file.strip() for file in testing_list]
    
with open(validation_list_path) as f:
    validation_list = f.readlines()
    validation_list = [file.strip() for file in validation_list]

In [5]:
command_dirs = os.listdir(dataset_dir)

command_dirs = [direc for direc in command_dirs if os.path.isdir(os.path.join(dataset_dir, direc))]

command_dirs.sort()

command_dirs.remove('_background_noise_')

In [6]:
print(command_dirs)

['bed', 'bird', 'cat', 'dog', 'down', 'eight', 'five', 'four', 'go', 'happy', 'house', 'left', 'marvin', 'nine', 'no', 'off', 'on', 'one', 'right', 'seven', 'sheila', 'six', 'stop', 'three', 'tree', 'two', 'up', 'wow', 'yes', 'zero']


In [7]:
key2label_dict = dict()
label2key_list = list()

key2label_dict[' '] = 0
label2key_list.append(' ')

for i, command in enumerate(command_dirs):
    key2label_dict[command] = i + 1
    label2key_list.append(command)

In [8]:
key2label_dict

{' ': 0,
 'bed': 1,
 'bird': 2,
 'cat': 3,
 'dog': 4,
 'down': 5,
 'eight': 6,
 'five': 7,
 'four': 8,
 'go': 9,
 'happy': 10,
 'house': 11,
 'left': 12,
 'marvin': 13,
 'nine': 14,
 'no': 15,
 'off': 16,
 'on': 17,
 'one': 18,
 'right': 19,
 'seven': 20,
 'sheila': 21,
 'six': 22,
 'stop': 23,
 'three': 24,
 'tree': 25,
 'two': 26,
 'up': 27,
 'wow': 28,
 'yes': 29,
 'zero': 30}

In [9]:
total_wav_file_list = list()

for command_dir in command_dirs:
    wav_list = os.listdir(os.path.join(dataset_dir, command_dir))
    
    wav_list_with_prefix = [os.path.join(command_dir, file_name) for file_name in wav_list]
    
    total_wav_file_list += wav_list_with_prefix

In [10]:
training_list = list(set(total_wav_file_list) - set(testing_list) - set(validation_list))
training_list.sort()


In [11]:
'''
validation_list
testing_list
training_list
'''

'\nvalidation_list\ntesting_list\ntraining_list\n'

In [12]:
audio_metadata = list()

for wav_file_name in total_wav_file_list:
    audio = AudioSegment.from_wav(dataset_dir + wav_file_name)
    fs = audio.frame_rate
    length_in_samples = audio.frame_count()
    
#   normalized_audio_array = np.asarray(audio.get_array_of_samples()) / 2 ** 15
    
#   print("Max: {}, Min: {}".format(max(normalized_audio_array), min(normalized_audio_array)))
    
    audio_metadata.append([fs, length_in_samples])

In [13]:
SR = 16000

class DatasetLoader():
    
    def __init__(self, batch_size, dataset_dir, wav_file_name_list, key2label, label2key):
        self.batch_size = batch_size
        self.dataset_dir = dataset_dir
        self.wav_file_name_list = copy.deepcopy(wav_file_name_list)
        self.sr = SR
        self.nsc_in_ms = 40
#         self.nov_in_ms = self.nsc_in_ms / 2
        self.nov_in_ms = 0
        self.nsc_in_sample = int(self.nsc_in_ms / 1000 * self.sr)
        self.nov_in_sample = int(self.nov_in_ms / 1000 * self.sr)
        
        self.key2label = key2label
        self.label2key = label2key
        
    def shuffle_dataset_order(self):
        random.shuffle(self.wav_file_name_list)
        
    def batch_generator(self):
        
        self.shuffle_dataset_order()
        
        spectrogram_list = list()
        label_list = list()
        
        for i, wav_file_name in enumerate(self.wav_file_name_list):
            
            audio = AudioSegment.from_wav(self.dataset_dir + wav_file_name)
            
            normalized_audio_array = np.asarray(audio.get_array_of_samples()) / 2 ** 15
    
            f, t, Zxx = scipy_signal.stft(normalized_audio_array, fs=self.sr, 
                                          nperseg=self.nsc_in_sample,
                                          noverlap=self.nov_in_sample)
        
            Sxx = np.abs(Zxx)
            
            normalized_spectrogram = (20 * np.log10(np.maximum(Sxx, 1e-8)) + 160) / 160
            spectrogram_list.append(normalized_spectrogram)
            
            keyword = wav_file_name.split('/')[0]
            label = self.key2label[keyword]
            label_list.append(label)
            

            if (i + 1) % self.batch_size == 0 or i + 1 == len(self.wav_file_name_list):
                
                spectrogram_time_step_list = [specgram.shape[1] for specgram in spectrogram_list]
                
                max_time_step = max(spectrogram_time_step_list)
                
                freq_size = spectrogram_list[0].shape[0]
                
                batch = np.zeros([len(spectrogram_list), freq_size, max_time_step])
                
                for j, specgram in enumerate(spectrogram_list):
                    batch[j, :specgram.shape[0], :specgram.shape[1]] = specgram
                
                spectrogram_list = list()
                
#                 print('{}:{}'.format(i, batch.shape[0]))
            
                yield batch, np.asarray(label_list)
                
                label_list = list()
                
#                 break
    
#             break

In [14]:
len(training_list)

51088

In [15]:
dataset_loader_training = DatasetLoader(256, dataset_dir, training_list, key2label_dict, label2key_list)
dataset_loader_testing = DatasetLoader(256, dataset_dir, testing_list, key2label_dict, label2key_list)
dataset_loader_validation = DatasetLoader(256, dataset_dir, validation_list, key2label_dict, label2key_list)

# dataset_loader_testing = DatasetLoader(64, dataset_dir, testing_list, key2label_dict, label2key_list)
# dataset_loader_validation = DatasetLoader(64, dataset_dir, validation_list, key2label_dict, label2key_list)

# dataset_loader_training = DatasetLoader(768, dataset_dir, training_list, key2label_dict, label2key_list)
# dataset_loader_testing = DatasetLoader(768, dataset_dir, testing_list, key2label_dict, label2key_list)
# dataset_loader_validation = DatasetLoader(768, dataset_dir, validation_list, key2label_dict, label2key_list)

In [16]:
# batch_generator = dataset_loader_training.batch_generator()

In [17]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [18]:
# class basic_model(nn.Module):
#     def __init__(self, D_in, H, num_layers, num_labels):
#         super(basic_model, self).__init__()
#         self.fc = torch.nn.Linear(D_in, H)
#         self.relu = torch.nn.ReLU()
#         self.dropout = nn.Dropout(p=0.2)

#         self.gru_layers = nn.ModuleList([nn.GRU(H, int(H / 2), bidirectional=True, batch_first=True) for i in range(num_layers)])

#         self.fc_pred = nn.Linear(H, num_labels)
#         self.log_softmax = nn.LogSoftmax(dim=2)
        
#     def forward(self, input_tensor):
#         # (B, T, F)
#         output_tensor = self.fc(input_tensor)
#         output_tensor = self.relu(output_tensor)
#         output_tensor = self.dropout(output_tensor)
#         # (B, T, H)
#         for layer in self.gru_layers:
#             output_tensor, _ = layer(output_tensor)
            
#         output_tensor = self.fc_pred(output_tensor)

#         output_tensor = self.log_softmax(output_tensor)
        
#         return output_tensor

class basic_conv_model(nn.Module):
#     def __init__(self, first_kernel_size, second_kernel_size, dropout_rate, num_labels):
    def __init__(self):
        super(basic_conv_model, self).__init__()
        self.conv2d_1 = nn.Conv2d(1, 16, (13, 2))
        self.conv2d_2 = nn.Conv2d(16, 32, (13, 2))
        self.flatten = nn.Flatten()
        self.fc = nn.Linear(4800, 31)
        
    def forward(self, input_tensor):
        input_tensor.unsqueeze_(1)
        tensor = self.conv2d_1(input_tensor)
#         print('[After 1st Conv2d]: {}'.format(tensor.shape))
        tensor = F.relu(tensor)
        tensor = F.dropout(tensor, 0.1, training=True)
        tensor = F.max_pool2d(tensor, (3, 2))
#         print('[After 1st maxpool2d]: {}'.format(tensor.shape))
        tensor = self.conv2d_2(tensor)
#         print('[After 2nd Conv2d]: {}'.format(tensor.shape))
        tensor = F.relu(tensor)
        tensor = F.dropout(tensor, 0.1, training=True)
        tensor = F.max_pool2d(tensor, (3, 2))
#         print('[After 2nd maxpool2d]: {}'.format(tensor.shape))
        tensor = self.flatten(tensor)
#         print('[After Flatten]: {}'.format(tensor.shape))
        tensor = self.fc(tensor)
#         print('[After fc]: {}'.format(tensor.shape))
        pred_tensor = F.log_softmax(tensor, dim=-1)
        
        return pred_tensor
    
class basic_crnn_model(nn.Module):
#     def __init__(self, first_kernel_size, second_kernel_size, dropout_rate, num_labels):
    def __init__(self):
        super(basic_crnn_model, self).__init__()
        self.conv2d = nn.Conv2d(1, 16, (13, 2))
        self.gru = nn.GRU(16 * 103, 128, 1)
        self.flatten = nn.Flatten()
        self.fc = nn.Linear(128 * 12, 31)
        
    def forward(self, input_tensor):
        input_tensor.unsqueeze_(1)  # (B, 1, F, T)
        tensor = self.conv2d(input_tensor) # (B, 16, F, T)
#         print('[After 1st Conv2d]: {}'.format(tensor.shape))
        tensor = F.relu(tensor)
        tensor = F.dropout(tensor, 0.1, training=True)
        tensor = F.max_pool2d(tensor, (3, 2))
#         print('[After 1st maxpool2d]: {}'.format(tensor.shape))
        
        tensor = tensor.view(tensor.shape[0], -1, tensor.shape[3])
        # (B, 16 * F, T)
        
#         print('[After Reshape]: {}'.format(tensor.shape))
        
        tensor.transpose_(0, 2) # (T, F, B)
        tensor.transpose_(1, 2) # (T, B, F)

        tensor, _ = self.gru(tensor)
#         print('[After 2nd Conv2d]: {}'.format(tensor.shape))
        tensor = F.relu(tensor)
        tensor = F.dropout(tensor, 0.1, training=True)
#         tensor = F.max_pool2d(tensor, (3, 2))
#         print('[After 2nd maxpool2d]: {}'.format(tensor.shape))
        tensor.transpose_(0, 1) # (B, T, F)
    
#         print('[After Reshape]: {}'.format(tensor.shape))
    
        tensor = self.flatten(tensor)
#         print('[After Flatten]: {}'.format(tensor.shape))
        tensor = self.fc(tensor)
#         print('[After fc]: {}'.format(tensor.shape))
        pred_tensor = F.log_softmax(tensor, dim=-1)
        
        return pred_tensor

    
# model = basic_model(321, 512, 3, 31).float().to(device)
# optimizer = optim.Adam(model.parameters(), lr=1e-4)
# dictation_loss = nn.CTCLoss().to(device)

model = basic_crnn_model().float().to(device)  
optimizer = optim.Adam(model.parameters(), lr=1e-4)
        

In [19]:
# batch_generator = dataset_loader_training.batch_generator()
# next(batch_generator)[0].shape

In [20]:
MAX_EPOCH = 300

for epoch in range(MAX_EPOCH):
    training_loss_list = list()
    batch_generator = dataset_loader_training.batch_generator()
    model.train()
    
    correct_num = 0
    total_num = 0

    for (batch, label) in batch_generator:
        optimizer.zero_grad()
        pred = model(torch.cuda.FloatTensor(batch))
        loss = F.nll_loss(pred, torch.cuda.LongTensor(label))
        loss.backward()
        training_loss_list.append(loss.item())
        optimizer.step()
        
        _, pred_label = torch.max(pred, 1)
        correct_num += int((pred_label == torch.cuda.LongTensor(label)).sum().cpu().numpy())
        total_num += len(label)
        
    acc_train = correct_num / total_num

    testing_loss_list = list()

    batch_generator = dataset_loader_testing.batch_generator()

    model.eval()
    
    correct_num = 0
    total_num = 0

    for (batch, label) in batch_generator:
    #     optimizer.zero_grad()
        pred = model(torch.cuda.FloatTensor(batch))
        loss = F.nll_loss(pred, torch.cuda.LongTensor(label))
#         loss.backward()
        testing_loss_list.append(loss.item())
    #     optimizer.step()
    
        _, pred_label = torch.max(pred, 1)
        correct_num += int((pred_label == torch.cuda.LongTensor(label)).sum().cpu().numpy())
        total_num += len(label)
        
    acc_test = correct_num / total_num

    validation_loss_list = list()

    batch_generator = dataset_loader_validation.batch_generator()
    
    correct_num = 0
    total_num = 0

    for (batch, label) in batch_generator:
    #     optimizer.zero_grad()
        pred = model(torch.cuda.FloatTensor(batch))
        loss = F.nll_loss(pred, torch.cuda.LongTensor(label))
#         loss.backward()
        validation_loss_list.append(loss.item())
    #     optimizer.step()
    
        _, pred_label = torch.max(pred, 1)
        correct_num += int((pred_label == torch.cuda.LongTensor(label)).sum().cpu().numpy())
        total_num += len(label)
        
    acc_valid = correct_num / total_num
    
    training_loss_mean = np.mean(training_loss_list)
    testing_loss_mean = np.mean(testing_loss_list)
    validation_loss_mean = np.mean(validation_loss_list)
    
    print(datetime.datetime.now().time())
    print('#{:03d} [Train: {:0.3f}] [Test: {:0.3f}] [Valid: {:0.3f}]'.format(epoch, training_loss_mean, testing_loss_mean, validation_loss_mean))
    print('#{:03d} [Train: {:0.3f}] [Test: {:0.3f}] [Valid: {:0.3f}]'.format(epoch, acc_train, acc_test, acc_valid))
    

07:31:12.420011
#000 [Train: 3.287] [Test: 3.068] [Valid: 3.029]
#000 [Train: 0.084] [Test: 0.138] [Valid: 0.145]
07:31:59.471162
#001 [Train: 2.912] [Test: 2.776] [Valid: 2.709]
#001 [Train: 0.171] [Test: 0.216] [Valid: 0.233]
07:32:45.450890
#002 [Train: 2.616] [Test: 2.519] [Valid: 2.448]
#002 [Train: 0.252] [Test: 0.288] [Valid: 0.314]
07:33:31.420872
#003 [Train: 2.377] [Test: 2.354] [Valid: 2.278]
#003 [Train: 0.322] [Test: 0.340] [Valid: 0.360]
07:34:17.680419
#004 [Train: 2.163] [Test: 2.134] [Valid: 2.056]
#004 [Train: 0.387] [Test: 0.394] [Valid: 0.420]
07:35:05.179339
#005 [Train: 1.971] [Test: 1.937] [Valid: 1.880]
#005 [Train: 0.444] [Test: 0.463] [Valid: 0.470]
07:35:52.449825
#006 [Train: 1.817] [Test: 1.838] [Valid: 1.802]
#006 [Train: 0.485] [Test: 0.477] [Valid: 0.483]
07:36:38.407999
#007 [Train: 1.675] [Test: 1.673] [Valid: 1.628]
#007 [Train: 0.532] [Test: 0.537] [Valid: 0.544]
07:37:24.236179
#008 [Train: 1.549] [Test: 1.586] [Valid: 1.539]
#008 [Train: 0.566] [Te

08:27:05.465489
#072 [Train: 0.286] [Test: 0.432] [Valid: 0.426]
#072 [Train: 0.921] [Test: 0.874] [Valid: 0.874]
08:27:51.770962
#073 [Train: 0.284] [Test: 0.428] [Valid: 0.437]
#073 [Train: 0.919] [Test: 0.873] [Valid: 0.875]
08:28:39.038711
#074 [Train: 0.280] [Test: 0.421] [Valid: 0.427]
#074 [Train: 0.921] [Test: 0.879] [Valid: 0.877]
08:29:25.170120
#075 [Train: 0.278] [Test: 0.442] [Valid: 0.438]
#075 [Train: 0.921] [Test: 0.869] [Valid: 0.870]
08:30:10.895271
#076 [Train: 0.273] [Test: 0.406] [Valid: 0.417]
#076 [Train: 0.923] [Test: 0.879] [Valid: 0.879]
08:30:57.434190
#077 [Train: 0.274] [Test: 0.426] [Valid: 0.417]
#077 [Train: 0.922] [Test: 0.873] [Valid: 0.875]
08:31:43.365267
#078 [Train: 0.268] [Test: 0.424] [Valid: 0.424]
#078 [Train: 0.924] [Test: 0.878] [Valid: 0.878]
08:32:29.266700
#079 [Train: 0.263] [Test: 0.407] [Valid: 0.409]
#079 [Train: 0.926] [Test: 0.883] [Valid: 0.880]
08:33:15.647662
#080 [Train: 0.262] [Test: 0.418] [Valid: 0.417]
#080 [Train: 0.926] [Te

09:22:53.640180
#144 [Train: 0.133] [Test: 0.377] [Valid: 0.389]
#144 [Train: 0.966] [Test: 0.895] [Valid: 0.893]
09:23:39.736932
#145 [Train: 0.131] [Test: 0.383] [Valid: 0.389]
#145 [Train: 0.965] [Test: 0.893] [Valid: 0.892]
09:24:28.441885
#146 [Train: 0.129] [Test: 0.378] [Valid: 0.384]
#146 [Train: 0.966] [Test: 0.895] [Valid: 0.894]
09:25:15.843541
#147 [Train: 0.129] [Test: 0.378] [Valid: 0.393]
#147 [Train: 0.966] [Test: 0.893] [Valid: 0.892]
09:26:02.462062
#148 [Train: 0.127] [Test: 0.388] [Valid: 0.389]
#148 [Train: 0.967] [Test: 0.893] [Valid: 0.893]
09:26:49.540152
#149 [Train: 0.124] [Test: 0.380] [Valid: 0.396]
#149 [Train: 0.968] [Test: 0.894] [Valid: 0.890]
09:27:36.095840
#150 [Train: 0.124] [Test: 0.385] [Valid: 0.387]
#150 [Train: 0.968] [Test: 0.894] [Valid: 0.893]
09:28:22.560927
#151 [Train: 0.123] [Test: 0.382] [Valid: 0.400]
#151 [Train: 0.968] [Test: 0.893] [Valid: 0.893]
09:29:09.956581
#152 [Train: 0.122] [Test: 0.398] [Valid: 0.388]
#152 [Train: 0.968] [Te

10:18:54.826310
#216 [Train: 0.061] [Test: 0.443] [Valid: 0.452]
#216 [Train: 0.986] [Test: 0.891] [Valid: 0.892]
10:19:41.559284
#217 [Train: 0.061] [Test: 0.433] [Valid: 0.434]
#217 [Train: 0.986] [Test: 0.893] [Valid: 0.893]
10:20:27.340969
#218 [Train: 0.060] [Test: 0.433] [Valid: 0.442]
#218 [Train: 0.986] [Test: 0.891] [Valid: 0.894]
10:21:13.208573
#219 [Train: 0.059] [Test: 0.435] [Valid: 0.446]
#219 [Train: 0.987] [Test: 0.890] [Valid: 0.895]
10:22:00.355689
#220 [Train: 0.059] [Test: 0.442] [Valid: 0.447]
#220 [Train: 0.987] [Test: 0.890] [Valid: 0.893]
10:22:46.492045
#221 [Train: 0.058] [Test: 0.430] [Valid: 0.439]
#221 [Train: 0.987] [Test: 0.891] [Valid: 0.894]
10:23:34.095206
#222 [Train: 0.057] [Test: 0.443] [Valid: 0.456]
#222 [Train: 0.988] [Test: 0.892] [Valid: 0.891]
10:24:20.476751
#223 [Train: 0.058] [Test: 0.433] [Valid: 0.442]
#223 [Train: 0.987] [Test: 0.895] [Valid: 0.896]
10:25:07.577978
#224 [Train: 0.056] [Test: 0.437] [Valid: 0.442]
#224 [Train: 0.987] [Te

11:14:22.016167
#288 [Train: 0.026] [Test: 0.533] [Valid: 0.540]
#288 [Train: 0.995] [Test: 0.891] [Valid: 0.892]
11:15:06.011405
#289 [Train: 0.025] [Test: 0.528] [Valid: 0.535]
#289 [Train: 0.996] [Test: 0.888] [Valid: 0.888]
11:15:50.512856
#290 [Train: 0.025] [Test: 0.503] [Valid: 0.534]
#290 [Train: 0.995] [Test: 0.890] [Valid: 0.893]
11:16:35.728519
#291 [Train: 0.024] [Test: 0.524] [Valid: 0.557]
#291 [Train: 0.996] [Test: 0.891] [Valid: 0.889]
11:17:20.326876
#292 [Train: 0.026] [Test: 0.528] [Valid: 0.544]
#292 [Train: 0.995] [Test: 0.892] [Valid: 0.890]
11:18:05.570654
#293 [Train: 0.024] [Test: 0.527] [Valid: 0.532]
#293 [Train: 0.996] [Test: 0.893] [Valid: 0.893]
11:18:50.422926
#294 [Train: 0.023] [Test: 0.528] [Valid: 0.555]
#294 [Train: 0.996] [Test: 0.889] [Valid: 0.891]
11:19:34.853312
#295 [Train: 0.023] [Test: 0.522] [Valid: 0.547]
#295 [Train: 0.996] [Test: 0.891] [Valid: 0.891]
11:20:18.976327
#296 [Train: 0.023] [Test: 0.523] [Valid: 0.559]
#296 [Train: 0.996] [Te

In [21]:
# label.shape

In [22]:
# output = model(torch.cuda.FloatTensor(batch))

In [23]:
# output.shape

In [24]:
# torch.tensor(batch).unsqueeze_(1).shape

In [25]:
# input_tensor = torch.tensor(batch).to(device).float()
# input_tensor = input_tensor.transpose(1, 2)
# model(input_tensor).shape

In [26]:
# for batch, label_list in batch_generator:
#     print(batch.shape[0])
#     plt.figure(figsize=(5, 1))
#     plt.plot(label_list)
#     plt.show()
    
#     plt.figure(figsize=(6, 18))
# #     plt.imshow(batch[0], aspect='auto')
#     plt.imshow(batch[0])
#     plt.show()
    
#     break